In [13]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop


In [14]:
# ==== Imports ====
import numpy as np
from gpie import Graph, ConstWave, SupportPrior, GaussianMeasurement, pmse
from gpie.core.linalg_utils import random_phase_mask, circular_aperture, random_normal_array

# ==== 1. パラメータ ====
H, W = 128, 128
shape = (H, W)
var = 1e-4
rng = np.random.default_rng(42)

# Known input X
x_ref = random_normal_array(shape, dtype=np.complex128, rng=rng)

# Support for unknown Y
support = circular_aperture(shape, radius=0.25)

# ==== 2. グラフ定義 ====
class MultiplyGraph(Graph):
    def __init__(self):
        super().__init__()
        X = ~ConstWave(data=x_ref, label="X_ref")
        Y = ~SupportPrior(support=support, label="Y_unknown")
        Z = X * Y  # MultiplyPropagator via __mul__
        with self.observe():
            meas = GaussianMeasurement(var=var) @ Z
        self.compile()

# ==== 3. 初期化と推論 ====
g = MultiplyGraph()
g.set_init_rng(np.random.default_rng(seed=1))
g.generate_sample(rng=np.random.default_rng(seed=123), update_observed=True)

true_y = g.get_wave("Y_unknown").get_sample()

def monitor(graph, t):
    est = graph.get_wave("Y_unknown").compute_belief().data
    err = pmse(est, true_y)
    print(f"[t={t}] PMSE = {err:.5e}")

g.run(n_iter=10, callback=monitor)



[t=0] PMSE = 1.39555e+00
[t=1] PMSE = 1.59020e-04
[t=2] PMSE = 1.59020e-04
[t=3] PMSE = 1.59020e-04
[t=4] PMSE = 1.59020e-04
[t=5] PMSE = 1.59020e-04
[t=6] PMSE = 1.59020e-04
[t=7] PMSE = 1.59020e-04
[t=8] PMSE = 1.59020e-04
[t=9] PMSE = 1.59020e-04


In [15]:
# ==== Imports ====
import numpy as np
from gpie import Graph, ConstWave, SupportPrior, GaussianMeasurement, pmse
from gpie.core.linalg_utils import random_phase_mask, circular_aperture, random_normal_array

# ==== 1. パラメータ ====
H, W = 128, 128
shape = (H, W)
var = 1e-4
large_value = 1e6
rng = np.random.default_rng(42)

# Known input X
x_ref = random_normal_array(shape, dtype=np.complex128, rng=rng)

# Support for unknown Y
support = circular_aperture(shape, radius=0.25)

# ==== 2. グラフ定義 ====
class MultiplyGraph(Graph):
    def __init__(self):
        super().__init__()
        Y = ~SupportPrior(support=support, label="Y_unknown")
        Z = x_ref * Y 
        with self.observe():
            meas = GaussianMeasurement(var=var) @ Z
        self.compile()

# ==== 3. 初期化と推論 ====
g = MultiplyGraph()
g.set_init_rng(np.random.default_rng(seed=1))
g.generate_sample(rng=np.random.default_rng(seed=123), update_observed=True)

true_y = g.get_wave("Y_unknown").get_sample()

def monitor(graph, t):
    est = graph.get_wave("Y_unknown").compute_belief().data
    err = pmse(est, true_y)
    print(f"[t={t}] PMSE = {err:.5e}")

g.run(n_iter=10, callback=monitor)


[t=0] PMSE = 1.00767e-03
[t=1] PMSE = 2.05594e-04
[t=2] PMSE = 2.05594e-04
[t=3] PMSE = 2.05594e-04
[t=4] PMSE = 2.05594e-04
[t=5] PMSE = 2.05594e-04
[t=6] PMSE = 2.05594e-04
[t=7] PMSE = 2.05594e-04
[t=8] PMSE = 2.05594e-04
[t=9] PMSE = 2.05594e-04


In [16]:
# ==== Imports ====
import numpy as np
from gpie import Graph, ConstWave, GaussianPrior, fft2, AmplitudeMeasurement, pmse
from gpie.core.linalg_utils import random_phase_mask, random_normal_array

# ==== 1. パラメータ ====
H, W = 128, 128
shape = (H, W)
var = 1e-4
rng = np.random.default_rng(123)

# ==== 2. グラフ定義 ====
class MixedMeasurementGraph(Graph):
    def __init__(self, num_masks):
        super().__init__()
        x = ~GaussianPrior(shape = shape, label = "object")
        for _ in range(num_masks):
            mask = ~ConstWave(data = random_normal_array(shape, rng=rng))
            with self.observe():
                measurement = AmplitudeMeasurement(var = var, damping = 0.3) @ (fft2(mask * x))
        self.compile()

# ==== 3. 推論・初期化 ====
g = MixedMeasurementGraph(num_masks = 4)
g.set_init_rng(np.random.default_rng(seed=999))
g.generate_sample(rng=np.random.default_rng(seed=100), update_observed=True)

true_w = g.get_wave("object").get_sample()

def monitor(graph, t):
    est = graph.get_wave("object").compute_belief().data
    err = pmse(est, true_w)
    if t % 20 == 0:
        print(f"[t={t}] PMSE = {err:.5e}")

g.run(n_iter=200, callback=monitor, verbose = None)

[t=0] PMSE = 1.08220e+00
[t=20] PMSE = 9.23174e-01
[t=40] PMSE = 2.21821e-03
[t=60] PMSE = 2.02464e-04
[t=80] PMSE = 2.02458e-04
[t=100] PMSE = 2.02459e-04
[t=120] PMSE = 2.02459e-04
[t=140] PMSE = 2.02459e-04
[t=160] PMSE = 2.02459e-04
[t=180] PMSE = 2.02459e-04


In [17]:
g.visualize()

Loading BokehJS ...